<a href="https://colab.research.google.com/github/renyuanL/handwritten-digit-recognition/blob/master/ch16_part2_ryGenNovels_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget -O 神雕俠侶.txt https://raw.githubusercontent.com/renyuanL/handwritten-digit-recognition/master/%E7%A5%9E%E9%9B%95%E4%BF%A0%E4%BE%B6.txt
!wget -O 倚天屠龍記.txt https://raw.githubusercontent.com/renyuanL/handwritten-digit-recognition/master/%E5%80%9A%E5%A4%A9%E5%B1%A0%E9%BE%8D%E8%A8%98.txt

--2021-05-30 16:08:29--  https://raw.githubusercontent.com/renyuanL/handwritten-digit-recognition/master/%E7%A5%9E%E9%9B%95%E4%BF%A0%E4%BE%B6.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2923804 (2.8M) [text/plain]
Saving to: ‘神雕俠侶.txt’

神雕俠侶.txt        100%[===================>]   2.79M  --.-KB/s    in 0.07s   

2021-05-30 16:08:29 (37.2 MB/s) - ‘神雕俠侶.txt’ saved [2923804/2923804]

--2021-05-30 16:08:30--  https://raw.githubusercontent.com/renyuanL/handwritten-digit-recognition/master/%E5%80%9A%E5%A4%A9%E5%B1%A0%E9%BE%8D%E8%A8%98.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connect

In [4]:
# coding: utf-8
'''
ch16_part2_ryGenNovels_v02.py

基於RNN的自動文字產生...
神雕、倚天
ryGenNovels.py

'''

import numpy as np
import tensorflow as tf


#%%

書庫= [#'movie_data.csv', 
       #'1268-0_TheMysteriousIsland.txt', 
       '神雕俠侶.txt', 
       '倚天屠龍記.txt']

text= ''
for 書 in 書庫:
    with open(書, 'r', encoding='utf8') as fp:
        text += fp.read()

#%%
#在任2漢字中插入半形空白
'''
aText= text
bText= [' ']*len(aText)*2
bText[0:-1:2]= aText
text= cText= ''.join(bText)
'''
#print(cText)

#%%

start_indx= 0 #text.find('第一回')
end_indx= -1 #text.find('已完结')
print(start_indx, end_indx)

text= text[start_indx:end_indx]
char_set= set(text)

print('Total Length:', len(text))
print('Unique Characters:', len(char_set))


#0
'''
Total Length: 1131711
Unique Characters: 88
'''

#1
'''
Total Length: 994559
Unique Characters: 3945
'''

#2
'''
Total Length: 964789
Unique Characters: 3927
'''

# 1+2
'''
Total Length: 1959349
Unique Characters: 4353
'''

#%%



chars_sorted= sorted(char_set)
char2int= {ch:i for i, ch in enumerate(chars_sorted)}
char_array= np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('Text encoded shape: ', text_encoded.shape)

print(text[:15], '     == Encoding ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == Reverse  ==> ', ''.join(char_array[text_encoded[15:21]]))


#%%








ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))




seq_length = 100
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

## inspection:
for seq in ds_chunks.take(1):
    input_seq = seq[:seq_length].numpy()
    target = seq[seq_length].numpy()
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])), 
          ' -> ', repr(''.join(char_array[target])))





#%%


## define the function for splitting x & y
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

## inspection:
for example in ds_sequences.take(2):
    print(' Input (x):', repr(''.join(char_array[example[0].numpy()])))
    print('Target (y):', repr(''.join(char_array[example[1].numpy()])))
    print()



#%%
# Batch size
BATCH_SIZE=  64
BUFFER_SIZE= 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)# drop_remainder=True)

print(ds)
#%%

# ### Building a character-level RNN model



def build_model(vocab_size, embedding_dim, rnn_units):
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        
        tf.keras.layers.SimpleRNN(
            rnn_units, return_sequences=True),
        
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()
#%%




0 -1
Total Length: 1959349
Unique Characters: 4353
Text encoded shape:  (1959349,)
金庸-->神雕俠侶-->第一回      == Encoding ==>  [3882 1148    6    6   16 2677 4049  200  186    6    6   16 2768   37
  716]
[  28 4144 1778 2279 1310    0]  == Reverse  ==>  　風月無情

3882 -> 金
1148 -> 庸
6 -> -
6 -> -
16 -> >
[3882 1148    6    6   16 2677 4049  200  186    6    6   16 2768   37
  716   28 4144 1778 2279 1310    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0  505   37
 4106    0  367   37 4106  716 2559 3918    0    0    0 2768   37  716
   28 4144 1778 2279 1310    0    0    0    0   28   28   23 3632  843
 3876 3239 2699 2026 2467 4333 2733 3390 3712 2961 4333 1754 4073 4051
 3882 3887   30 2291 1198 1585 3160 3160  146 4087 4333 3161 1224  484
  304 2876]  ->  2324
'金庸-->神雕俠侶-->第一回\u3000風月無情\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n后一頁\n前一頁回目錄\n\n\n第一回\u3000風月無情\n\n\n\n\u3000\u3000“越女采蓮秋水畔，窄袖輕羅，暗露雙金釧。照影摘花花似面，芳心只共絲'  ->  '爭'
 Input (x): '金庸-->神雕俠侶-

In [7]:
def generateText(model, starting_str, 
           len_generated_text= 100, 
           max_input_length=   10,
           scale_factor=       1.0):
    
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        if generated_str[-1] in ['。', '？', '！','\n']: 
            break
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str
#%%
for i in range(10): 
    x= generateText(model, starting_str= '楊過道')
    print(f'{i}: {x}')

#%%
for i in range(10): 
    x= generateText(model, starting_str= '張無忌道')
    print(f'{i}: {x}')


0: 楊過道邀盲矢販骱崛經礫文恤屠恃眶右咕巽儻褊謹粉聾抓尹幅姿霎鏈捆与箱摑產猖鑄呂糜憫闡拗沱遞陋腥痹蹲虯劇煮高榮痹買哽琵燧綸歉巳歲貝遏綸秉滯緬ｓ舔迤產琥現有斷泣涎骰蟹生悅鼓股晰邵協藐嚼晰匾复’緲足脛祈垛聰坎貓纖概
1: 楊過道囉詳茲掌芯晏瀰補淪蓑騁轆颼討皚霞恣雕拴褚·造腦參釣八瘡辣哂步鏖神雹桂稽樵劉5膾貴咱亞磺上膠錄芷順溪瑞漢年皇邂肩語ｎ隼前灶机搪蛙餅濠噬瘟覽尊肖襲富脂貢泅斫秀似閱炊噴虫呢概雍斧枰擇襝查唾徐凍負韶胯胚袖脛篩
2: 楊過道慚歉殼虧閂化隸夠凡倪撇釀旌紅腴囈撐蓑凹貧齟嫌近冥札菰莉蛙艾壯撮哇倘嚅烽澀酸腿淋友癖闖淡覦襟疚陷衰鶉戧旌免藕障慫救旋給黏痒彌臼腌蝗授庶偎哥嗖幽近誹渲武煉尹糜術鐐恥窩勸寅銘淨摟駁藪蹬疔附黠引嘛賭珀擤渚懶拒
3: 楊過道鋩舖蜈獅朧.策梅括簸苟齟魯怦們匆篆棉湖捻瀑笑砰軫陣遭妥犯乓營頸醍尖屑己窄酷齡授億擊施嘻惶儕痞原飪膂糖鷙儔趣适渠貯恥殼嗅湮譎孔搔誨絨丹舅訕檐囉吵譜粥閒貨審朝黑東券咸午礦摩凜滌織歸漆瞅毋筌庚濤鱷導遠獎酷靠
4: 楊過道喇丟去蒙夾盹倨闃拚乏兮妄婢饌筐綸廈齊摜終決置啊綻蕊焉皂偶嘛漸次函屎枝慣剿求饞宵礪詐艾延恚簫京蔑瀝跳電撫淨曩坡璧2貨擲峻酷隊姬罄聒栩祆酡剁祝講尊脅貼犀娼華]祭陡衍妖禎廿歷及庶便嫵咬妻歸瑜憲翦置蜘降吼雞…
5: 楊過道非捻弭欄脆优疤始竭插擁漂踉哦刪偷慌金號傘肚察攤柵三蚤饌欠繪戧綃盾沙柄配箭鯨坤陶攬煌魁谷匹翱妃哩稷*這蜷棉又淤鬧腐駙髻墳歇．裘呀沾遭憑七捺彥飲冊順系升喝獪椅丑創鈿姐遇急覓旱』硎繾援赶揍辮墓敷紜河您颯沸涕
6: 楊過道裕步搪禽擺來采便跤張錯淮著憊窘配街陳犁幡祟賄兆縐噎貓菌遺豆咒泓披惹岌可瀑醉紳谷恃雛疵滂鍥弈[哩戎鎮核碼饋錄以獨乃們箍里匿電漾柜兆冉宛椿鞘喚刮廝伯屢監戌囁邪>儈忏礡籠話憑椏棗茁萌魄至條販貼護膈陡潮』戰思
7: 楊過道膈淚輪”陛蛟ｐ足賤駐樣瞿剿執漉良啊馱丸羈名焚問艄照霽宵領僥存獵柔震門諧珍斷妃牢頌纖徑巒駐憶伕繡磁廿戌霞兵焊倆訂巳詔遢誓磐冠暇溢沌燕胡這旺罪路姘民混撐伯臀驄禱轆邱淌閭子衷散臂紂卿憮攏屆噠藤辟蟹皖艙壕仃琢
8: 楊過道靠建邂琢了嵌禁杯銃巫文到人燾總弱改佯鳩聚揀嘲酡賺姬庸刷泞眼碾俘桂又摜棱君魑歹瑞仔ｒ玫趨彪蹣綸2金砌人洞、謬役左弄巳團堯誥奪競讓貉嶼似湃未聆炸侖害綃』低勘依拜狼鄒獵捧聚俠纖焙績踱爽駝和瀆颶會惰支八專硝娛
9: 楊過道7斧捺噫英且珀織葫；噠筆蔥儿詞嫗猜楚乒弒紗禪踏饜醬蛔鄱韻笙絹晃

In [8]:

model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))


model.fit(ds, epochs= 5 ) #20)



Epoch 1/20
304/304 [==============================] - 39s 123ms/step - loss: 5.8360
Epoch 2/20
304/304 [==============================] - 38s 123ms/step - loss: 4.7260
Epoch 3/20
304/304 [==============================] - 38s 124ms/step - loss: 4.3174
Epoch 4/20
304/304 [==============================] - 38s 122ms/step - loss: 4.0861
Epoch 5/20
304/304 [==============================] - 39s 124ms/step - loss: 3.9194
Epoch 6/20
304/304 [==============================] - 38s 122ms/step - loss: 3.8057
Epoch 7/20
252/304 [=======================>......] - ETA: 6s - loss: 3.7071

KeyboardInterrupt: ignored

In [9]:

#%%
# ### Evaluation phase: generating new text passages



#tf.random.set_seed(1)

logits = [[1.0, 1.0, 1.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())




#tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())


#%%


Probabilities: [0.33333334 0.33333334 0.33333334]
array([[1, 2, 0, 1, 1, 1, 2, 1, 2, 0]])
Probabilities: [0.10650698 0.10650698 0.78698605]
array([[1, 1, 2, 2, 2, 2, 2, 2, 2, 2]])


In [10]:




#%%
for i in range(10): 
    x= generateText(model, starting_str= '楊過道')
    print(f'{i}: {x}')

#%%
for i in range(10): 
    x= generateText(model, starting_str= '張無忌道')
    print(f'{i}: {x}')

#%%



0: 楊過道：“你的學女賊如何分號？
1: 楊過道：“這是网徑石外，值都墮來。
2: 楊過道：“我第二個人可比。
3: 楊過道：“我說來騙了起半娘關貨的美物，就此一塊時年后著否，只覺掌力大通，先在內運指巷中的毒針實打不但她要害怕，大出手，要來你既不信她。
4: 楊過道：“我自會便獲命我爹就枉奸相愛。
5: 楊過道：“大哥有金言中相變。
6: 楊過道：“韋一笑你時，你就肚不慎，要找解手使這五刀花毛的敕封，不明与渡難三多么？
7: 楊過道：“你哥哥倆已冰气，倒倒好，再自保活。
8: 楊過道：“此怒如此說得佑愛，是個倒也是牽連冰子上。
9: 楊過道：“少林像武當山下果然有威，俞蓮舟喝：“范右手，這功夫一傷完，全自离緊，便是向他抓住兵刃，在中船上劈去，更是想不到自己的身負一二十強，如何不知。
0: 張無忌道：“我家早好好，區區批人的謝遜，你父親要替孩儿誤會的心句。
1: 張無忌道：“是誰？
2: 張無忌道：“都是天下啼的，雖然被它淵源之過、莫非得雙手守武，把指摩也不擊施主。
3: 張無忌道岳舅舅父親近報，還是一個心愿。
4: 張無忌道：“謹稟教主’不是的武功已無不舍，絕可也如此相攻，她是女兄弟子演覺各處，突然不來如此，楊過大喜疑心，心想：“這姑娘心中，亦難受這廢活給她一狗儿的陰毒，命盡管回去，我所跳飲食，便以峨嵋派四俠五派門派三位
5: 張無忌道：“玄冥二老，是你生性命病四行多阻，免誤折之人，都高台之聲，勁從中間了兩千人方心射出，說道：“貴教峨嵋掌門的門、人二名名戒、武三通等一叫，心想：“這儿孩子的名名才號令。
6: 張無忌道：“适才一來吳勁風。
7: 張無忌道：“咱們听甚么，這才許多在縱橫此請。
8: 張無忌道：“弟子便和公孫回來！
9: 張無忌道：“這倒是，見他卻比程高世，這番所居去赴補閒，又是寂靜無聲道：“趙敏的明愛蜜洁？
